In [1]:
import graphlab as gl

#needed for charts
import matplotlib.pyplot as plt
%matplotlib inline

This non-commercial license of GraphLab Create is assigned to s104065891@gapp.nthu.edu.tw and will expire on March 29, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-04-03 20:20:10,843 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1459686008.log


# Loading Data

In [2]:
# loading the train and test data
customers = gl.SFrame('train.csv/train.csv')
test_sframe = gl.SFrame('test.csv/test.csv')

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/train.csv/train.csv

Parsing completed. Parsed 100 lines in 3.23178 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,float,float,float,float,int,int,float,float,float,int,int,float,int,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,float,int,float,float,int,int,int,int,int,float,int,float,float,float,float,int,int,int,float,float,int,int,int,float,float,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,i

Read 67150 lines. Lines per second: 14305.9

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/train.csv/train.csv

...

Parsing completed. Parsed 100 lines in 2.03968 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,float,float,int,int,int,int,int,float,float,float,float,float,float,float,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,int,float,float,float,int,int,float,int,int,int,int,float,float,float,int,int,int,float,float,float,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int

Read 67325 lines. Lines per second: 16987.7

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/test.csv/test.csv

Parsing completed. Parsed 75818 lines in 4.29174 secs.

# Exploring the Data

In [3]:
#gl.canvas.set_target('ipynb')
#customers['TARGET'].show(view='Categorical')
#customers.show()

In [4]:
#remove all columns with constant columns; where STD == 0
remove = []

for col in customers.column_names():
    if customers[col].std() == 0:
        remove.append(col)

customers.remove_columns(remove)
customers.num_columns()

337

### Most Target are 0, which means most customers are satisfied

In [5]:
train_data, test_data = customers.random_split(.8, seed=2)

In [8]:
test_data.select_columns(['TARGET'])

TARGET
0
0
0
0
0
0
0
0
0
0


In [9]:
len(test_data[test_data['TARGET'] == 1])

577

In [10]:
customer_model = gl.boosted_trees_classifier.create(train_data, target='TARGET', max_depth=100, metric='auc')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: Detected extremely low variance for feature(s) 'delta_imp_reemb_var33_1y3', 'delta_num_reemb_var33_1y3', 'imp_reemb_var17_hace3', 'imp_reemb_var33_ult1', 'imp_trasp_var17_in_hace3', 'num_reemb_var17_hace3', 'num_reemb_var33_ult1', 'num_trasp_var17_in_hace3' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 57720

...

| 7         | 24.082745    | 0.960620     | 0.841173       |

| 8         | 27.801785    | 0.975883     | 0.839830       |

| 9         | 30.832441    | 0.979618     | 0.836918       |

| 10        | 34.630984    | 0.985506     | 0.835051       |

+-----------+--------------+--------------+----------------+

In [18]:
customer_model.evaluate(test_data)

{'accuracy': 0.9582296713369124,
 'auc': 0.8025087117957136,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |   21  |
 |      0       |        1        |   82  |
 |      1       |        0        |  556  |
 |      0       |        0        | 14615 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.06176470588235294,
 'log_loss': 0.1477484430736988,
 'precision': 0.20388349514563106,
 'recall': 0.036395147313691506,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-----+-------+
 | threshold | fpr | tpr |  p  |   n   |
 +-----------+-----+-----+-----+-------+
 |    0.0    | 1.0 | 1.0 | 577 | 14697 |
 |   1e-05   | 1.0 | 1.0 | 577 | 14697 |
 |   2e-05   | 

## Save Predictions

In [19]:
prediction = customer_model.predict(test_sframe, output_type='probability')

In [20]:
prediction

dtype: float
Rows: 75818
[0.034795477986335754, 0.05801544338464737, 0.023656826466321945, 0.03091331757605076, 0.023656826466321945, 0.1096838191151619, 0.09830135107040405, 0.48653653264045715, 0.1036754697561264, 0.05104504153132439, 0.2412354052066803, 0.023656826466321945, 0.061280399560928345, 0.07822214812040329, 0.024924499914050102, 0.040527913719415665, 0.29365912079811096, 0.023656826466321945, 0.03754127025604248, 0.05521395802497864, 0.02867504023015499, 0.07541035860776901, 0.02769504114985466, 0.025366542860865593, 0.1707194298505783, 0.02553410455584526, 0.023656826466321945, 0.023656826466321945, 0.0825151801109314, 0.03091331757605076, 0.04782845824956894, 0.023656826466321945, 0.024924499914050102, 0.027663491666316986, 0.02973874844610691, 0.0653790831565857, 0.02973874844610691, 0.02733723260462284, 0.08724197000265121, 0.04771745204925537, 0.023656826466321945, 0.16947680711746216, 0.023656826466321945, 0.023656826466321945, 0.024924499914050102, 0.024924499914050

In [21]:
#function for making submission
def make_submission(prediction, filename='submission.txt'):
    with open(filename, 'w') as f:
        #header
        f.write('ID,TARGET\n')
        submission_strings = test_sframe['ID'].astype(str) + ',' + prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n')

make_submission(prediction, 'submission6.txt')


## Test with XGB Boost here

In [12]:
import xgboost as xgb

In [13]:
clf = xgb.XGBClassifier(max_depth=5, n_estimators=350, learning_rate=0.03, nthread=4, 
                       subsample=0.95, colsample_bytree=0.85, seed=4242)

In [14]:
# create list of featured not including the TARGET
features = [f for f in train_data.column_names() if f != 'TARGET']

In [15]:
len(features)

336

In [17]:
clf.fit(train_data.select_columns(features), train_data.select_columns(['TARGET']), early_stopping_rounds=20, eval_metric="auc", 
        eval_set=[(test_data.select_columns(features),test_data.select_columns(['TARGET']))])

TypeError: can not initialize DMatrix from SFrame